In [26]:
import pandas as pd
from EcoNameTranslator import to_scientific, to_common, to_species
import os
import csv
from pytaxize import scicomm
from pygbif import species

In [27]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 20)


In [28]:

usdaDF = pd.read_csv('G:/Shared drives/host_tree_cnn/cleaning_species_names/addl_data/usda_code.txt')
print(usdaDF.columns.tolist())
usdaDF['Scientific Name with Author'] = usdaDF['Scientific Name with Author'].str.lower()
usdaDF['Common Name'] = usdaDF['Common Name'].str.lower()
print(usdaDF.head(100))

['Symbol', 'Synonym Symbol', 'Scientific Name with Author', 'Common Name', 'Family']
    Symbol Synonym Symbol Scientific Name with Author          Common Name  \
0     ABAB            NaN  abutilon abutilo...         shrubby indian m...   
1     ABAB          ABAM5  abutilon america...                         NaN   
2     ABAB           ABJA  abutilon jacquin...                         NaN   
3     ABAB           ABLI  abutilon lignosu...                         NaN   
4   ABAB70            NaN  abietinella abie...            abietinella moss   
5   ABAB70           HYAB  hypnum abietinum...                         NaN   
6   ABAB70         THAB70  thuidium abietin...                         NaN   
7     ABAL            NaN  abronia alpina b...         ramshaw meadows ...   
8    ABAL3            NaN     abies alba mill.                  silver fir   
9     ABAM            NaN  abies amabilis (...          pacific silver fir   
10   ABAM2            NaN  abronia ameliae ...         am

In [29]:
# ['Symbol', 'Synonym Symbol', 'Scientific Name with Author', 'Common Name', 'Family']
prevFindings = {}
# clean up the scientific names
def cleanScientific(scientificName):
    global prevFindings
    global iterations
    if (scientificName == 'NA'):
        iterations += 1
        return ["NA", "NA"]
    # if iterations % 100 == 0 or iterations == 1:
    #     print(str(i) + "/" + str(numRows))
    if scientificName in prevFindings:
        return prevFindings[scientificName]
    try:
        index = to_species([scientificName])
        values = index[scientificName][0].split()
        if len(values) > 2:
            print(values)
        prevFindings[scientificName] = [values[-2], values[-1]]
        return values[-2], values[-1]
    except:
        prevFindings[scientificName] = ["NA", "NA"]
        return ["NA", "NA"]


In [30]:
# scientific name to common name
noneCount = 0
sNaValues = set()
scientificNamesDict = {}
def getCommonNames(scientificName):
    global noneCount
    global sNaValues

    if scientificName in sNaValues:
        noneCount += 1
        return None
    elif scientificName in scientificNamesDict:
        return scientificNamesDict[scientificName]
    try:
        common_names = to_common([scientificName])[scientificName][1]
        if len(common_names) == 0:
            raise ValueError()
        
        scientificNamesDict[scientificName] = common_names
        
        return common_names
    except:
        noneCount += 1 
        sNaValues.add(scientificName)
        return None

In [31]:
# common name to scientific Name

print(usdaDF.columns.tolist())
# ['Symbol', 'Synonym Symbol', 'Scientific Name with Author', 'Common Name', 'Family']

cNaValues = set()
common_namesDict = {}
def getScientificNames(common_name):
    global noneCount
    global cNaValues

    common_name = common_name.lower()
    print(common_name)


    if common_name in cNaValues:
        noneCount += 1
        return None
    elif common_name in common_namesDict:
        print(common_namesDict[common_name])
        return common_namesDict[common_name]
    try:
        sci_name = to_scientific([common_name])[common_name][1]
        if len(sci_name) == 0:
            raise ValueError()
        
        common_namesDict[common_name] = sci_name[0]
        print(sci_name[0])
        return sci_name[0]
    except:
      result = usdaDF.loc[usdaDF['Common Name'] == common_name, 'Scientific Name with Author'].values
      if len(result) > 0:
          print("RESULT")
          print(result[0])
          common_namesDict[common_name] = result[0]
          return result[0]

      print("NONE")
      noneCount += 1 
      cNaValues.add(common_name)
      return "NA"

['Symbol', 'Synonym Symbol', 'Scientific Name with Author', 'Common Name', 'Family']


In [32]:
symbolDict = symbolDF['Scientific Name with Author'].to_dict()
synonymDict = synonymDF['Scientific Name with Author'].to_dict()

sciDict = symbolDict.copy()  
sciDict.update(synonymDict)
valid_codes = {
    "ABBA": "Abies balsamea",
    "JUVI": "Juniperus virginiana",
    "CAOV": "Carya ovata",
    "FRAM": "Fraxinus americana",
    "ACNI": "Acer nigrum",
    "CAGL": "Carya glabra",
    "QUCO": "Quercus coccinea",
    "QUMA1": "Quercus macrocarpa",
    "POTR1": "Populus tremuloides",
    "ULWI": "Ulmus wilsoniana"
}
valid2 = {
    "ACCA": "Acer campestre",
    "ACNI": "Acer nigrum",
    "ACPA": "Acer palmatum",
    "ACPL": "Acer platanoides",
    "ACRU": "Acer rubrum",
    "ACSA1": "Acer saccharinum",
    "ACSA2": "Acer saccharum",
    "AECA": "Aesculus californica",
    "AMLA": "Amelanchier laevis",
    "BENI": "Betula nigra",
    "CABE": "Carpinus betulus",
    "CAGL": "Carya glabra",
    "CAOV": "Carya ovata",
    "CEOC": "Celtis occidentalis",
    "CHLA": "Chamaecyparis lawsoniana",
    "CHOB": "Chionanthus virginicus",
    "CRMO": "Crataegus monogyna",
    "EUAT": "Eucommia ulmoides",
    "FASY": "Fagus sylvatica",
    "FRAM": "Fraxinus americana",
    "FREX": "Fraxinus excelsior",
    "FRNI": "Fraxinus nigra",
    "FROR": "Fraxinus ornus",
    "FRPE": "Fraxinus pennsylvanica",
    "GIBI": "Ginkgo biloba",
    "GLTR": "Gleditsia triacanthos",
    "GYDI": "Gymnocladus dioicus",
    "HAVI": "Hamamelis virginiana",
    "ILCO": "Ilex coriaceae",
    "JUSC": "Juglans cinerea",
    "LIST": "Liquidambar styraciflua",
    "NYSY": "Nyssa sylvatica",
    "PONI": "Populus nigra",
    "POTR": "Populus tremuloides",
    "PRSE": "Prunus serotina",
    "PRSU": "Prunus subhirtella",
    "PRVI": "Prunus virginiana",
    "PYCA": "Pyrus calleryana",
    "QUCO": "Quercus coccinea",
    "QUMA": "Quercus macrocarpa",
    "QUPA": "Quercus palustris",
    "QURO": "Quercus robur",
    "ROPS": "Robinia pseudoacacia",
    "SEGI": "Sequoiadendron giganteum",
    "SOJA": "Sophora japonica",
    "SYRE": "Syringa reticulata",
    "TADI": "Taxodium distichum",
    "THOC": "Thuja occidentalis",
    "THPL": "Thuja plicata",
    "TIAM": "Tilia americana",
    "TICO": "Tilia cordata",
    "TIPL": "Tilia platyphyllos",
    "ULAM": "Ulmus americana",
    "ULPA": "Ulmus parvifolia",
    "ULWI": "Ulmus wilsoniana",
    "ZESE": "Zelkova serrata"
}

def get_species_name(code):
    # First check if the full code is in the dictionary
    if code in sciDict:
        return sciDict[code]
    # If not, check the first four letters of the code
    else:
        return sciDict.get(code[:4], None)


In [33]:
# reverse comma ordering
def reorganizeComma(name):
    if isinstance(name, str):

        if ',' in name:
            parts = name.split(',', 1)
            preComma = parts[0].strip()
            postComma = parts[1].strip()
            return postComma + ' ' + preComma
        else:
            return name
    else:
        return name

In [34]:
# Handle NA Values
def handleNA(value):
    value = value.replace('.', '')
    # questions --- common name as array, or string?
    naValues = {'', 'a', 'nan', 'other', 'n/a', ' ', 'not specified', 'na', 'none', 'p', 'f'}
    containsNaValues = {'not specified', 'unidentified', 'unsuitable', 'vacant', '*', '_', '-', 'proposed', 'unknown', '#', 'other ', 'no ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

    if value.strip() in naValues or any(word in value for word in containsNaValues):
        return 'NA'
    else:
        # If the value contains there characters, include only the elements before
        if "'" in value:
            return value.split("'")[0]
        elif "(" in value:
            return value.split("(")[0]
        elif ":" in value:
            return value.split(":")[0]
        elif "`" in value:
            return value.split("`")[0]
        elif "‘" in value:
            return value.split("‘")[0]

        return value

In [35]:





    


# ==================================================================================================================================
# Get city and create columns

# Find the folder path and create a dataframe from it
folder_path = "G:/Shared drives/host_tree_cnn/cleaning_species_names/og_inventories_modified_labels"
# currentCities = {"columbus_inventory.csv"}
# Iterate through the files

for i, filename in enumerate(os.listdir(folder_path)):
    if filename == 'usda_code.csv':
        continue
    if filename != 'columbus_inventory.csv':
        continue
    # if filename not in currentCities:
    #     continue
    iterations = 1
    print(filename)
    print("FILE #", i)
    file_path = os.path.join(folder_path, filename)
    if not filename.endswith('.csv'):
        continue

    cityDF = pd.read_csv(file_path)
    
    cityDF = cityDF.map(reorganizeComma)
    
    
    if filename == 'columbus_inventory.csv':
        symbolDict = {}
        synonymDict = {}
        symbolDF = usdaDF.set_index('Symbol')
        synonymDF = usdaDF.set_index('Synonym Symbol')

        symbolDict = symbolDF['Common Name'].to_dict()
        synonymDict = synonymDF['Common Name'].to_dict()

        commonDict = symbolDict.copy()  
        commonDict.update(synonymDict)


        symbolDict = symbolDF['Scientific Name with Author'].to_dict()
        synonymDict = synonymDF['Scientific Name with Author'].to_dict()

        sciDict = symbolDict.copy()  
        sciDict.update(synonymDict)
        valid_codes = {
            "ABBA": "Abies balsamea",
            "JUVI": "Juniperus virginiana",
            "CAOV": "Carya ovata",
            "FRAM": "Fraxinus americana",
            "ACNI": "Acer nigrum",
            "CAGL": "Carya glabra",
            "QUCO": "Quercus coccinea",
            "QUMA1": "Quercus macrocarpa",
            "POTR1": "Populus tremuloides",
            "ULWI": "Ulmus wilsoniana"
        }
        valid2 = {
            "ACCA": "Acer campestre",
            "ACNI": "Acer nigrum",
            "ACPA": "Acer palmatum",
            "ACPL": "Acer platanoides",
            "ACRU": "Acer rubrum",
            "ACSA1": "Acer saccharinum",
            "ACSA2": "Acer saccharum",
            "AECA": "Aesculus californica",
            "AMLA": "Amelanchier laevis",
            "BENI": "Betula nigra",
            "CABE": "Carpinus betulus",
            "CAGL": "Carya glabra",
            "CAOV": "Carya ovata",
            "CEOC": "Celtis occidentalis",
            "CHLA": "Chamaecyparis lawsoniana",
            "CHOB": "Chionanthus virginicus",
            "CRMO": "Crataegus monogyna",
            "EUAT": "Eucommia ulmoides",
            "FASY": "Fagus sylvatica",
            "FRAM": "Fraxinus americana",
            "FREX": "Fraxinus excelsior",
            "FRNI": "Fraxinus nigra",
            "FROR": "Fraxinus ornus",
            "FRPE": "Fraxinus pennsylvanica",
            "GIBI": "Ginkgo biloba",
            "GLTR": "Gleditsia triacanthos",
            "GYDI": "Gymnocladus dioicus",
            "HAVI": "Hamamelis virginiana",
            "ILCO": "Ilex coriaceae",
            "JUSC": "Juglans cinerea",
            "LIST": "Liquidambar styraciflua",
            "NYSY": "Nyssa sylvatica",
            "PONI": "Populus nigra",
            "POTR": "Populus tremuloides",
            "PRSE": "Prunus serotina",
            "PRSU": "Prunus subhirtella",
            "PRVI": "Prunus virginiana",
            "PYCA": "Pyrus calleryana",
            "QUCO": "Quercus coccinea",
            "QUMA": "Quercus macrocarpa",
            "QUPA": "Quercus palustris",
            "QURO": "Quercus robur",
            "ROPS": "Robinia pseudoacacia",
            "SEGI": "Sequoiadendron giganteum",
            "SOJA": "Sophora japonica",
            "SYRE": "Syringa reticulata",
            "TADI": "Taxodium distichum",
            "THOC": "Thuja occidentalis",
            "THPL": "Thuja plicata",
            "TIAM": "Tilia americana",
            "TICO": "Tilia cordata",
            "TIPL": "Tilia platyphyllos",
            "ULAM": "Ulmus americana",
            "ULPA": "Ulmus parvifolia",
            "ULWI": "Ulmus wilsoniana",
            "ZESE": "Zelkova serrata"
        }

        sciDict.update(valid_codes)
        sciDict.update(valid2)
        cityDF['common_name'] = cityDF['SP_CODE'].map(commonDict)
        cityDF['species_name'] = cityDF['SP_CODE'].map(get_species_name)
        
        print(cityDF[['SP_CODE', 'common_name', 'species_name']].head(100))
        print("Percents")
        na_percentage = cityDF['common_name'].isna().mean() * 100
        print(na_percentage)

        na_percentage = cityDF['species_name'].isna().mean() * 100
        print(na_percentage)
        na_percentage = cityDF['SP_CODE'].isna().mean() * 100
        print(na_percentage)
        null_species_codes = cityDF[cityDF['species_name'].isna()]['SP_CODE'].drop_duplicates()
        null_species_codes_set = set(null_species_codes)
        print(null_species_codes_set)

    # Drop all na values and format the table to a string
    # cityDF.dropna(how='all')
    cityDF['unique_common_name'] = cityDF['common_name'].astype(str)

    # In case you need to get scientific names from ecotranslator
    if filename == 'bloomington_inventory.csv':
      print(cityDF[['unique_common_name']])
      cityDF['species_name'] = cityDF['unique_common_name'].apply(getScientificNames)
      print(cityDF)
    cityDF['unique_sciname'] = cityDF['species_name'].astype(str)

    # handle space removal and lowercase conversion
    cityDF['unique_sciname'] = cityDF['unique_sciname'].str.strip() 
    cityDF['unique_sciname'] = cityDF['unique_sciname'].str.lower()

    cityDF['unique_common_name'] = cityDF['unique_common_name'].str.strip()
    cityDF['unique_common_name'] = cityDF['unique_common_name'].str.lower()

    cityDF['unique_sciname'] = cityDF['unique_sciname'].map(handleNA)
    cityDF['unique_common_name'] = cityDF['unique_common_name'].map(handleNA)
    


    cityDF[['genus_name', 'species_name']] = cityDF.apply(
        lambda row: pd.Series(cleanScientific(row['unique_sciname'])),
        axis=1,
        result_type='expand'
    )
    cols = [col for col in cityDF.columns if col not in ['unique_common_name', 'unique_sciname', 'genus_name', 'species_name']]
    # Append the new columns at the end
    cols += ['unique_common_name', 'unique_sciname', 'genus_name', 'species_name']

    # Reorder the DataFrame
    cityDF = cityDF[cols]





    cityDF.to_csv("G:/Shared drives/host_tree_cnn/cleaning_species_names/og_inventories_w_names_appended/" + filename, index=False)



columbus_inventory.csv
FILE # 6


C:\Users\brenn\AppData\Local\Temp\ipykernel_32\3120028469.py:23: DtypeWarning: Columns (3,6,8,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  cityDF = pd.read_csv(file_path)


   SP_CODE          common_name         species_name
0     QUPA           hybrid oak    Quercus palustris
1    ACSA2                  NaN       Acer saccharum
2     PLAC                  NaN  allocarya acanth...
3     PLAC                  NaN  allocarya acanth...
4     PLAC                  NaN  allocarya acanth...
5     PLAC                  NaN  allocarya acanth...
6     PLAC                  NaN  allocarya acanth...
7     QUSH        shumard's oak  quercus shumardi...
8     LIST                  NaN  Liquidambar styr...
9     ACPL                  NaN     Acer platanoides
10    HISY       rose of sharon  hibiscus syriacu...
11    ULPR                  NaN  ulmus minor auct...
12    ACPL                  NaN     Acer platanoides
13   ACSA2                  NaN       Acer saccharum
14    QURU     northern red oak     quercus rubra l.
15    ACRU                  NaN          Acer rubrum
16    ACRU                  NaN          Acer rubrum
17     MA2                  NaN               